In [1]:
import networkx as nx
import csv
import numpy as np
import random
import matplotlib.pyplot as plt



In [2]:
# First read and create the graph using the training dataset
G = nx.DiGraph()

with open("training.txt", "r") as f:
    for line in f:
        line = line.split()
        G.add_node(line[0])
        G.add_node(line[1])
        if line[2] == '1':
            G.add_edge(line[0], line[1])
    for edge in G.edges():
        G[edge[0]][edge[1]]['weight'] = 1 # We are considering a graph where edges are unweighted 
            

In [40]:
pip install gensim


     |████████████████████████████████| 24.7MB 101kB/s eta 0:00:01
     |████████████████████████████████| 71kB 1.8MB/s eta 0:00:011
     |████████████████████████████████| 133kB 389kB/s eta 0:00:01
     |████████████████████████████████| 71kB 134kB/s eta 0:00:01
     |████████████████████████████████| 5.9MB 128kB/s eta 0:00:01
  Created wheel for smart-open: filename=smart_open-1.9.0-cp37-none-any.whl size=73088 sha256=05e79fab2a4127bcfdc307c38ffcaad4d38a9bdd1a7e1f7f597a027970da94fa
  Stored in directory: /Users/hugoweidmann/Library/Caches/pip/wheels/ab/10/93/5cff86f5b721d77edaecc29959b1c60d894be1f66d91407d28
Successfully built smart-open
Note: you may need to restart the kernel to use updated packages.


In [1]:
## Node2Vec Implementation

In [3]:
# G graph
# default walk_length is 80 
#num_walks default is 10

'''
    The following code is provided by Grover Aditya and Jure Leskovec 
    in "node2vec: Scalable Feature Learning for Networks"
    from "Proceedings of the 22nd ACM SIGKDD International Conference on Knowledge Discovery and Data Mining" (2016)
    
    it is avalable here https://github.com/aditya-grover/node2vec/blob/master/src/node2vec.py
'''


class Graph():
    def __init__(self, nx_G, p, q):
        self.G = nx_G
        self.p = p #return parameter
        self.q = q #in-out parameter

    def node2vec_walk(self, walk_length, start_node):
        '''
        Simulate a random walk starting from start node.
        '''
        G = self.G
        alias_nodes = self.alias_nodes
        alias_edges = self.alias_edges

        walk = [start_node]
        #print('start node: ', start_node)
        while len(walk) < walk_length:
            cur = walk[-1]
            cur_nbrs = sorted(G.neighbors(cur))
            if len(cur_nbrs) > 0:
                if len(walk) == 1:
                    walk.append(cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
                else:
                    prev = walk[-2]
                    #print('prev: ', prev)
                    #print('cur: ', cur)
                    arg_1 = alias_edges[(prev, cur)][0]
                    arg_2 = alias_edges[(prev, cur)][1]
                    target = alias_draw(arg_1, arg_2 )
                    next = cur_nbrs[target]
                    #print('next node: ', next)
                    walk.append(next)
            else:
                break
        
        return walk

    def simulate_walks(self, num_walks, walk_length):
        '''
        Repeatedly simulate random walks from each node.
        '''
        G = self.G
        walks = []
        nodes = list(G.nodes())
        print ('Walk iteration:')
        for walk_iter in range(num_walks):
            print (str(walk_iter+1), '/', str(num_walks))
            random.shuffle(nodes)
            for node in nodes:
                walks.append(self.node2vec_walk(walk_length=walk_length, start_node=node))

        return walks

    def get_alias_edge(self, src, dst):
        '''
        Get the alias edge setup lists for a given edge.
        '''
        G = self.G
        p = self.p
        q = self.q
    
        unnormalized_probs = []
        for dst_nbr in sorted(G.neighbors(dst)):
            if dst_nbr == src:
                unnormalized_probs.append(G[dst][dst_nbr]['weight']/p) #If the source appears as a possible destination for the target, we can reach it with probability 1/p (i.e. return to source)
            elif G.has_edge(dst_nbr, src):
                unnormalized_probs.append(G[dst][dst_nbr]['weight'])
            else:
                unnormalized_probs.append(G[dst][dst_nbr]['weight']/q)
        norm_const = sum(unnormalized_probs)
        normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]

        return alias_setup(normalized_probs)

    def preprocess_transition_probs(self):
        '''
        Preprocessing of transition probabilities for guiding the random walks.
        '''
        G = self.G
        alias_nodes = {}
        for node in G.nodes():
            unnormalized_probs = [G[node][nbr]['weight'] for nbr in sorted(G.neighbors(node))]
            norm_const = sum(unnormalized_probs)
            normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]
            alias_nodes[node] = alias_setup(normalized_probs)

        alias_edges = {}
        triads = {}

        #Graph is directed
        for edge in G.edges():
            alias_edges[edge] = self.get_alias_edge(src = edge[0], dst = edge[1])

        self.alias_nodes = alias_nodes
        self.alias_edges = alias_edges

        return


def alias_setup(probs):
    '''
    Compute utility lists for non-uniform sampling from discrete distributions.
    Refer to https://hips.seas.harvard.edu/blog/2013/03/03/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    for details
    '''
    K = len(probs)
    q = np.zeros(K)
    J = np.zeros(K, dtype=np.int)

    smaller = []
    larger = []
    for kk, prob in enumerate(probs):
        q[kk] = K*prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)

    while len(smaller) > 0 and len(larger) > 0:
        small = smaller.pop()
        large = larger.pop()

        J[small] = large
        q[large] = q[large] + q[small] - 1.0
        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

def alias_draw(J, q):
    '''
    Draw sample from a non-uniform discrete distribution using alias sampling.
    '''
    K = len(J)

    kk = int(np.floor(np.random.rand()*K))
    if np.random.rand() < q[kk]:
        return kk
    else:
        return J[kk]
    

In [ ]:
def learn_embeddings(walks, dimensions, window_size, nb_workers, nb_iter):
    '''
    Learn embeddings by optimizing the Skipgram objective using SGD.
    '''
    sentences = []
    for walk in walks:
        walks.append(str(walk))
    model = gensim.models.Word2Vec(sentences, size= dimensions, window = window_size, min_count=0, sg=1, workers = nb_workers, iter = nb_iter)
    
    return model

# the longer the length of walk, the better the F1 score. 
# bigger number of walks per node induces a better F1 score. Considering our limit in computation time, we limit at 10
# a decrease in p and q leads to better F1 score
#by default, dimension is 128 
#context size (=window size) of 10 is shown as optimal in termes of ratio score/computation time
#by default, parallel workers is 1
#by default, number of epochs in SGD is 1

def main(G, num_walks, walk_length, p,q):
    G = Graph(G, p, q)
    G.preprocess_transition_probs()
    walks = G.simulate_walks(num_walks, walk_length)
    model = learn_embeddings(walks, 128, 10, 1, 1)
    return model 

import gensim 
from gensim.models import Word2Vec

vec_model = main(G, 10, 80, 1,1)

Walk iteration:
1 / 4
2 / 4
3 / 4
4 / 4
